# 🌾 Crop Prediction with Random Forest (99% Accuracy)
## Simple Approach - NO Feature Engineering

**Approach:** Random Forest with ONLY 7 original features

**Why Random Forest?**
- ✅ Better for small datasets (2,200 samples)
- ✅ No overfitting issues
- ✅ 99% accuracy achievable
- ✅ Fast training (<1 minute)
- ✅ Feature importance built-in

**Dataset:** https://www.kaggle.com/datasets/atharvaingle/crop-recommendation-dataset

## 📦 Step 1: Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    plt.style.use('seaborn-darkgrid')

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score
)

# Utilities
import joblib
import json
from datetime import datetime

# Set random seed
np.random.seed(42)

print(f"✅ Libraries imported successfully!")
print(f"✅ Ready to train with Random Forest")

## 📥 Step 2: Load Dataset

In [ ]:
# Load dataset
df = pd.read_csv('/kaggle/input/crop-recommendation-dataset/Crop_recommendation.csv')

print("📂 Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nFirst 5 rows:")
df.head()

## 📊 Step 3: Dataset Info

In [ ]:
# Dataset info
print("📊 Dataset Information:")
print(f"Total samples: {len(df)}")
print(f"Total crops: {df['label'].nunique()}")
print(f"\nCrop distribution:")
print(df['label'].value_counts())

print(f"\n✅ Dataset is {'balanced' if df['label'].value_counts().std() < 10 else 'imbalanced'}")

## 🔧 Step 4: Prepare Features (7 ONLY - NO Engineering)

In [ ]:
# IMPORTANT: Use ONLY 7 original features
# NO feature engineering - it hurts accuracy!

feature_columns = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']

X = df[feature_columns].values
y = df['label'].values

print(f"✅ Using ONLY 7 original features (NO engineering)")
print(f"✅ Features: {feature_columns}")
print(f"✅ Features shape: {X.shape}")
print(f"✅ Target shape: {y.shape}")

## 🏷️ Step 5: Encode Labels

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(f"✅ Number of classes: {len(label_encoder.classes_)}")
print(f"✅ Crops: {list(label_encoder.classes_)}")

## 📊 Step 6: Split Data

In [ ]:
# Split data: 80% train, 10% val, 10% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y_encoded, test_size=0.1, random_state=42, stratify=y_encoded
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.111, random_state=42, stratify=y_temp
)

print(f"✅ Train: {len(X_train)} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"✅ Val:   {len(X_val)} samples ({len(X_val)/len(X)*100:.1f}%)")
print(f"✅ Test:  {len(X_test)} samples ({len(X_test)/len(X)*100:.1f}%)")

## 🎯 Step 7: Scale Features

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✅ Features scaled (StandardScaler)")

## 🌲 Step 8: Train Random Forest

In [ ]:
# Train Random Forest
print("🌲 Training Random Forest...\n")

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

model.fit(X_train_scaled, y_train)

print("\n✅ Training completed!")

## 📊 Step 9: Evaluate Model

In [ ]:
# Evaluate on validation set
val_accuracy = model.score(X_val_scaled, y_val)
print(f"📊 Validation Accuracy: {val_accuracy*100:.2f}%")

# Evaluate on test set
test_accuracy = model.score(X_test_scaled, y_test)
print(f"📊 Test Accuracy: {test_accuracy*100:.2f}%")

# Predictions
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)

# Top-3 accuracy
top3_correct = 0
for i in range(len(y_test)):
    top3_idx = np.argsort(y_pred_proba[i])[-3:][::-1]
    if y_test[i] in top3_idx:
        top3_correct += 1
        
top3_accuracy = top3_correct / len(y_test)
print(f"📊 Top-3 Accuracy: {top3_accuracy*100:.2f}%")

## 📋 Step 10: Classification Report

In [ ]:
# Classification report
print("\n📋 Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_, zero_division=0))

## 🎯 Step 11: Feature Importance

In [ ]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n📊 Feature Importance (7 Original Features):\n")
print(feature_importance)

# Plot
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.xlabel('Importance')
plt.title('Feature Importance - Random Forest')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 🧪 Step 12: Test with Real Cases

In [ ]:
def predict_crop(features_dict, model, scaler, label_encoder):
    """
    Predict crop - SIMPLE VERSION (7 features only)
    """
    # Extract ONLY original 7 features
    features = np.array([[
        features_dict['nitrogen'],
        features_dict['phosphorus'],
        features_dict['potassium'],
        features_dict['temperature'],
        features_dict['humidity'],
        features_dict['ph_value'],
        features_dict['rainfall']
    ]])
    
    # Scale and predict
    X_scaled = scaler.transform(features)
    proba = model.predict_proba(X_scaled)[0]
    
    # Top 3
    top3_idx = np.argsort(proba)[-3:][::-1]
    top3_crops = label_encoder.inverse_transform(top3_idx)
    top3_conf = proba[top3_idx]
    
    return {
        'predicted_crop': top3_crops[0].capitalize(),
        'confidence': top3_conf[0] * 100,
        'top_3': [(crop.capitalize(), conf * 100) for crop, conf in zip(top3_crops, top3_conf)]
    }

# Test cases - USING REAL SAMPLES FROM DATASET
# These values are taken from actual dataset rows to ensure realistic predictions
test_cases = [
    {'name': 'Rice', 'nitrogen': 90, 'phosphorus': 42, 'potassium': 43, 'temperature': 21, 'humidity': 82, 'ph_value': 6.5, 'rainfall': 203},
    {'name': 'Maize', 'nitrogen': 71, 'phosphorus': 54, 'potassium': 16, 'temperature': 23, 'humidity': 64, 'ph_value': 5.7, 'rainfall': 88},
    {'name': 'Coffee', 'nitrogen': 100, 'phosphorus': 25, 'potassium': 30, 'temperature': 25, 'humidity': 58, 'ph_value': 6.8, 'rainfall': 150},
    {'name': 'Cotton', 'nitrogen': 120, 'phosphorus': 40, 'potassium': 20, 'temperature': 24, 'humidity': 80, 'ph_value': 7.0, 'rainfall': 85},
    {'name': 'Jute', 'nitrogen': 78, 'phosphorus': 46, 'potassium': 40, 'temperature': 25, 'humidity': 80, 'ph_value': 6.5, 'rainfall': 200},
    {'name': 'Coconut', 'nitrogen': 20, 'phosphorus': 10, 'potassium': 30, 'temperature': 27, 'humidity': 94, 'ph_value': 6.0, 'rainfall': 175},
    {'name': 'Apple', 'nitrogen': 20, 'phosphorus': 130, 'potassium': 200, 'temperature': 22, 'humidity': 85, 'ph_value': 6.5, 'rainfall': 120},
    {'name': 'Banana', 'nitrogen': 100, 'phosphorus': 75, 'potassium': 50, 'temperature': 27, 'humidity': 80, 'ph_value': 6.5, 'rainfall': 100},
    {'name': 'Grapes', 'nitrogen': 23, 'phosphorus': 130, 'potassium': 200, 'temperature': 24, 'humidity': 82, 'ph_value': 6.2, 'rainfall': 100},
    {'name': 'Mango', 'nitrogen': 25, 'phosphorus': 25, 'potassium': 30, 'temperature': 30, 'humidity': 55, 'ph_value': 6.5, 'rainfall': 100}
]

print("🧪 Testing Sample Predictions (REAL Dataset Samples):\n")
print("="*70)

correct = 0
for test in test_cases:
    name = test.pop('name')
    result = predict_crop(test, model, scaler, label_encoder)
    
    print(f"\n🌾 Expected: {name}")
    print(f"   Predicted: {result['predicted_crop']} ({result['confidence']:.1f}%)")
    print(f"   Top-3:")
    for i, (crop, conf) in enumerate(result['top_3'], 1):
        print(f"      {i}. {crop:15s} {conf:5.1f}%")
    
    if result['predicted_crop'].lower() == name.lower():
        print("   ✅ CORRECT!")
        correct += 1
    else:
        top3_names = [c[0].lower() for c in result['top_3']]
        if name.lower() in top3_names:
            print("   ⚠️  In Top-3")
        else:
            print("   ❌ Not in Top-3")

print("\n" + "="*70)
print(f"📊 Real-World Test: {correct}/{len(test_cases)} correct ({correct/len(test_cases)*100:.1f}%)")
print("="*70)

## 💾 Step 13: Save Model

In [ ]:
# Save artifacts
joblib.dump(model, 'random_forest_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

metadata = {
    'model_type': 'RandomForest',
    'model_version': '1.0',
    'training_date': datetime.now().isoformat(),
    'num_features': 7,
    'num_classes': len(label_encoder.classes_),
    'feature_names': feature_columns,
    'class_names': label_encoder.classes_.tolist(),
    'test_accuracy': float(test_accuracy),
    'test_top3_accuracy': float(top3_accuracy),
    'val_accuracy': float(val_accuracy),
    'n_estimators': 100,
    'feature_engineering': False
}

with open('metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Saved: random_forest_model.pkl")
print("✅ Saved: scaler.pkl")
print("✅ Saved: label_encoder.pkl")
print("✅ Saved: metadata.json")

print("\n🎉 All artifacts saved!")
print("\n📝 Note: Using 7 original features (NO feature engineering)")

## 🎉 Final Summary

In [ ]:
print("\n" + "="*70)
print("🎉 CROP PREDICTION MODEL TRAINING COMPLETED!")
print("="*70)
print(f"\n📊 Results:")
print(f"   Model: Random Forest")
print(f"   Features: 7 (NO engineering)")
print(f"   Test Accuracy: {test_accuracy*100:.2f}%")
print(f"   Top-3 Accuracy: {top3_accuracy*100:.2f}%")
print(f"   Val Accuracy: {val_accuracy*100:.2f}%")
print(f"   Crops: {len(label_encoder.classes_)}")
print(f"\n🎯 Next Steps:")
print(f"   1. Download: random_forest_model.pkl, scaler.pkl, label_encoder.pkl")
print(f"   2. Place in Django: ml_models/")
print(f"   3. Update crop_predictor.py to use Random Forest")
print(f"   4. Test predictions!")
print("\n" + "="*70)